# 使用 Caching

網站如果有支援快取功能，透過 HTTP response 中的：

    Cache-Control,
    Etag,
    Date,
    Expires,
    Vary

等標頭，告知我們資訊，以減少不必要的爬取次數。 

以 example.com/index.html 為例：

```html
HTTP/1.1 200 OK 
Accept-Ranges: bytes 
Cache-Control: max-age=604800 
Content-Type: text/html; charset=UTF-8 
Date: Mon, 29 Oct 2018 13:31:23 GMT 
Etag: "1541025663" 
Expires: Mon, 05 Nov 2018 13:31:23 GMT 
Last-Modified: Fri, 09 Aug 2013 23:54:35 GMT 
Server: ECS (dca/53DB) 
Vary: Accept-Encoding 
X-Cache: HIT 
Content-Length: 1270 
...
```

## Cache-Control

此 header 指出內容是否支援快取，以及多久的時間。

常見數值有：

    * no-cache
    * no-store
    * must-revalidated
    * max-age=<秒數>
    * public
    
前三項防止客戶端(Client)使用快取；如果伺服器沒有指定此三項數值，我們可以根據 max-age 來進行快取更新。

經過 max-age 秒數後，網頁內容可能會有更動，可進行爬取。

## Expires

另一個可指出維持快取多久的標頭。

此例中，它告訴我們七天後，網頁內容可能有異，可進行快取。

```html
Date: Mon, 29 Oct 2018 13:31:23 GMT
Expires: Mon, 05 Nov 2018 13:31:23 GMT
```

## Etag

此網頁獨特的數值，只有當網頁內容有異動時，會被伺服器端重新給予新的數值。

我們可以透過發送 `If-None-Match` header 和 Etag 值給伺服端，伺服端會檢查此 Etag 值是否與網頁的 Etag 一致，如果**一致**，代表內容__未更動__，回應 Client 一個狀態碼: **304 Not Modified**. 如此我們便可以省下重新爬取網頁的動作與時間。

```html
HTTP/1.1 304 Not Modified 
Accept-Ranges: bytes 
Cache-Control: max-age=604800 
Date: Fri, 02 Nov 2018 14:37:16 GMT 

Etag: "1541025663" 

Expires: Fri, 09 Nov 2018 14:37:16 GMT 
Last-Modified: Fri, 09 Aug 2013 23:54:35 GMT 
Server: ECS (dca/53DB) 
Vary: Accept-Encoding 
X-Cache: HIT 
```

如果網頁有新內容，伺服端會回傳新的 Etag 與 Expires 時間，我們便可根據此數值更新我們的爬蟲。

# Caching content in Go

套件：

1. httpcache
2. diskv

## httpcache

    go get github.com/gregjones/httpcache

* 提供本機端儲存、擷取網頁功能
* 可自動處理快取版的 HTTP 請求與回應
* 支援多種資料庫後端，如：Redis, Memcached, LevelDB

## diskv

    go get github.com/peterbourgon/diskv

In [1]:
import(
    "fmt"
    "log"
    "io/ioutil"
    "github.com/gregjones/httpcache"
    "github.com/gregjones/httpcache/diskcache"
)

In [6]:
// 範例：有使用快取
{
    // 1. 於本機端建立快取檔 cache
    locStorage := diskcache.New("./cache")
    cache := httpcache.NewTransport(locStorage)
    
    // True: 通知我們，回應內容是否由快取中讀取
    cache.MarkCachedResponses = true
    cacheClient := cache.Client()
    
    // 建立初始請求
    fmt.Println("Caching: www.example.com/index.html")
    resp, err := cacheClient.Get("http://www.example.com/index.html")
    if err != nil {
        log.Fatalln(err)
    }
    
    /* DEBUG
    fmt.Println("Header:")
    for k, v := range resp.Header {
        fmt.Printf("%15q\t", k)
        fmt.Println(v)
    }
    */
    
    // httpcache 需要讀取網頁回應的 body 內容
    ioutil.ReadAll(resp.Body)
    /* DEBUG
    content, _ := ioutil.ReadAll(resp.Body)
    fmt.Println("Body:\n", string(content))
    */
    resp.Body.Close()
    
    // 再一次請求 index.html
    fmt.Println("Caching: www.example.com/index.html")
    resp, err = cacheClient.Get("http://www.example.com/index.html")
    if err != nil {
        log.Fatalln(err)
    }
    
    // 查看 httpcache 加入的旗標是否從快取中被讀取
    _, ok := resp.Header["X-From-Cache"]
    if ok {
        fmt.Println("Result was pulled from the cache!")
    }
    
    // DEBUG
    fmt.Println("Header:")
    for k, v := range resp.Header {
        fmt.Printf("%15q\t", k)
        fmt.Println(v)
    }
}

Caching: www.example.com/index.html
Caching: www.example.com/index.html
Result was pulled from the cache!
Header:
"Cache-Control"	[max-age=604800]
 "Content-Type"	[text/html; charset=UTF-8]
         "Date"	[Fri, 16 Aug 2019 13:13:49 GMT]
         "Etag"	["1541025663+gzip"]
      "Expires"	[Fri, 23 Aug 2019 13:13:49 GMT]
"Last-Modified"	[Fri, 09 Aug 2013 23:54:35 GMT]
      "X-Cache"	[HIT]
       "Server"	[ECS (oxr/8321)]
         "Vary"	[Accept-Encoding]
 "X-From-Cache"	[1]


In [7]:
// 範例：沒有使用快取
{
    // 1. 於本機端建立快取檔 cache
    locStorage := diskcache.New("./cache")
    cache := httpcache.NewTransport(locStorage)
    
    // True: 通知我們，回應內容是否由快取中讀取
    cache.MarkCachedResponses = true
    cacheClient := cache.Client()
    
    // 建立初始請求
    // fmt.Println("Caching: www.example.com/index.html")
    // resp, err := cacheClient.Get("http://www.example.com/index.html")
    fmt.Println("Caching: https://www.coursera.org/")
    resp, err := cacheClient.Get("https://www.coursera.org/")
    if err != nil {
        log.Fatalln(err)
    }
    
    /* DEBUG
    fmt.Println("Header:")
    for k, v := range resp.Header {
        fmt.Printf("%15q\t", k)
        fmt.Println(v)
    }
    */
    
    // httpcache 需要讀取網頁回應的 body 內容
    ioutil.ReadAll(resp.Body)
    /* DEBUG
    content, _ := ioutil.ReadAll(resp.Body)
    fmt.Println("Body:\n", string(content))
    */
    resp.Body.Close()
    
    // 再一次請求 index.html
    fmt.Println("Caching: https://www.coursera.org/")
    resp, err = cacheClient.Get("https://www.coursera.org/")
    if err != nil {
        log.Fatalln(err)
    }
    
    // 查看 httpcache 加入的旗標是否從快取中被讀取
    _, ok := resp.Header["X-From-Cache"]
    if ok {
        fmt.Println("Result was pulled from the cache!")
    }
    
    // DEBUG
    fmt.Println("Header:")
    for k, v := range resp.Header {
        fmt.Printf("%25q\t", k)
        fmt.Println(v)
    }
}

Caching: https://www.coursera.org/
Caching: https://www.coursera.org/
Header:
                   "Etag"	[W/"3f803-Q6t2wMXQA44GPvxWkDhUP12CelY"]
"X-Coursera-Trace-Id-Hex"	[d282d77016ab7318]
           "X-Powered-By"	[Express]
                "X-Cache"	[Miss from cloudfront]
            "X-Amz-Cf-Id"	[c0FsRhwe4QiXTio65b_LWS3nVyu40KAWYK1OeuDrXWsiZG3a_Txi2w==]
           "Content-Type"	[text/html]
"Strict-Transport-Security"	[max-age=31536000; includeSubDomains; preload]
 "X-Coursera-Render-Mode"	[html]
"X-Envoy-Upstream-Service-Time"	[516]
       "X-Xss-Protection"	[1; mode=block]
                   "Date"	[Fri, 16 Aug 2019 13:14:14 GMT]
                   "Vary"	[Accept-Encoding]
"X-Coursera-Render-Version"	[v2]
        "X-Frame-Options"	[SAMEORIGIN]
                    "Via"	[1.1 055b7c72fa08d18532a096f1c33f6fd0.cloudfront.net (CloudFront)]
           "X-Amz-Cf-Pop"	[TPE51-C1]
          "Cache-Control"	[private, no-cache, no-store, must-revalidate, max-age=0]
                 "Server"	[